# Import libraries

In [1]:
# Advanced Domain Adversarial Neural Network (DANN) for Text Generation Detection
# This notebook implements an optimized DANN model for detecting machine-generated vs human-written text across different domains.

import json
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
import random
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Loading data and preprocessing

In [2]:
def load_json_data(path):
    """Load training data from JSON file and convert to text format for vectorization"""
    texts, labels = [], []
    with open(path, 'r') as f:
        for line in f:
            sample = json.loads(line)
            # Convert token indices to string for vectorizer
            token_str = ' '.join(map(str, sample['text']))
            texts.append(token_str)
            labels.append(sample['label'])
    return texts, labels

def load_test_data(path):
    """Load test data and return texts with IDs"""
    test_texts, test_ids = [], []
    with open(path, 'r') as f:
        for line in f:
            sample = json.loads(line)
            text = ' '.join(map(str, sample['text']))
            test_texts.append(text)
            test_ids.append(sample['id'])
    return test_texts, test_ids

# Load datasets
print("Loading training datasets...")
X1, y1 = load_json_data('domain1_train_data.json')
X2, y2 = load_json_data('domain2_train_data.json')

print(f"Domain 1: {len(X1)} samples, Class distribution: {np.bincount(y1)}")
print(f"Domain 2: {len(X2)} samples, Class distribution: {np.bincount(y2)}")

Loading training datasets...
Domain 1: 1000 samples, Class distribution: [500 500]
Domain 2: 5000 samples, Class distribution: [ 250 4750]


# High level of feature engineering

In [3]:
# Split domains into train/validation
X1_train, X1_val, y1_train, y1_val = train_test_split(X1, y1, test_size=0.2, random_state=42, stratify=y1)
X2_train, X2_val, y2_train, y2_val = train_test_split(X2, y2, test_size=0.2, random_state=42, stratify=y2)

# Advanced vectorization with both Count and TF-IDF features
print("Creating advanced feature vectors...")

# Count vectorizer for basic term frequency
count_vectorizer = CountVectorizer(max_features=10000, ngram_range=(1, 2))
X_train_combined = X1_train + X2_train
count_vectorizer.fit(X_train_combined)

# TF-IDF vectorizer for weighted features
tfidf_vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
tfidf_vectorizer.fit(X_train_combined)

# Transform training data
X1_train_count = count_vectorizer.transform(X1_train)
X1_train_tfidf = tfidf_vectorizer.transform(X1_train)
X2_train_count = count_vectorizer.transform(X2_train)
X2_train_tfidf = tfidf_vectorizer.transform(X2_train)

# Combine count and TF-IDF features
from scipy.sparse import hstack
X1_train_vec = hstack([X1_train_count, X1_train_tfidf])
X2_train_vec = hstack([X2_train_count, X2_train_tfidf])

# Transform validation data
X1_val_vec = hstack([count_vectorizer.transform(X1_val), tfidf_vectorizer.transform(X1_val)])
X2_val_vec = hstack([count_vectorizer.transform(X2_val), tfidf_vectorizer.transform(X2_val)])

print(f"Feature dimensions: {X1_train_vec.shape[1]}")

Creating advanced feature vectors...
Feature dimensions: 15000


# SMOTE to deal with the imbalanced dataset

In [4]:
# Apply SMOTE to balance Domain 2 class distribution
print("Applying SMOTE to balance Domain 2...")
smote = SMOTE(random_state=42, k_neighbors=3)
X2_train_balanced, y2_train_balanced = smote.fit_resample(X2_train_vec.toarray(), y2_train)

print(f"Domain 2 after SMOTE: {len(y2_train_balanced)} samples, Class distribution: {np.bincount(y2_train_balanced)}")

# Convert to numpy arrays for PyTorch
X1_train_np = X1_train_vec.toarray().astype(np.float32)
X2_train_np = X2_train_balanced.astype(np.float32)
X1_val_np = X1_val_vec.toarray().astype(np.float32)
X2_val_np = X2_val_vec.toarray().astype(np.float32)

y1_train_np = np.array(y1_train, dtype=np.int64)
y2_train_np = np.array(y2_train_balanced, dtype=np.int64)
y1_val_np = np.array(y1_val, dtype=np.int64)
y2_val_np = np.array(y2_val, dtype=np.int64)

Applying SMOTE to balance Domain 2...
Domain 2 after SMOTE: 7600 samples, Class distribution: [3800 3800]


# Structure of the DNN model

In [5]:
class GradientReversalLayer(torch.autograd.Function):
    """Gradient Reversal Layer for adversarial training"""
    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha
        return x
    
    @staticmethod
    def backward(ctx, grad_output):
        return -ctx.alpha * grad_output, None

class AdvancedDANN(nn.Module):
    """Advanced Domain Adversarial Neural Network with improved architecture"""
    
    def __init__(self, input_dim, hidden_dims=[512, 256, 128], num_classes=2, num_domains=2, dropout_rate=0.5):
        super(AdvancedDANN, self).__init__()
        
        # Advanced feature extractor with residual connections
        self.feature_extractor = nn.ModuleList()
        prev_dim = input_dim
        
        for hidden_dim in hidden_dims:
            self.feature_extractor.append(nn.Sequential(
                nn.Linear(prev_dim, hidden_dim),
                nn.BatchNorm1d(hidden_dim),
                nn.ReLU(),
                nn.Dropout(dropout_rate)
            ))
            prev_dim = hidden_dim
        
        # Residual connection
        self.residual_projection = nn.Linear(input_dim, hidden_dims[-1])
        
        # Label classifier with attention mechanism
        self.label_classifier = nn.Sequential(
            nn.Linear(hidden_dims[-1], hidden_dims[-1] // 2),
            nn.BatchNorm1d(hidden_dims[-1] // 2),
            nn.ReLU(),
            nn.Dropout(dropout_rate * 0.5),
            nn.Linear(hidden_dims[-1] // 2, num_classes)
        )
        
        # Domain classifier
        self.domain_classifier = nn.Sequential(
            nn.Linear(hidden_dims[-1], hidden_dims[-1] // 2),
            nn.BatchNorm1d(hidden_dims[-1] // 2),
            nn.ReLU(),
            nn.Dropout(dropout_rate * 0.5),
            nn.Linear(hidden_dims[-1] // 2, num_domains)
        )
        
        # Attention mechanism for features
        self.attention = nn.Sequential(
            nn.Linear(hidden_dims[-1], hidden_dims[-1]),
            nn.Tanh(),
            nn.Linear(hidden_dims[-1], 1),
            nn.Sigmoid()
        )
    
    def forward(self, x, alpha=1.0):
        # Feature extraction with residual connection
        features = x
        for layer in self.feature_extractor:
            features = layer(features)
        
        # Add residual connection
        residual = self.residual_projection(x)
        features = features + residual
        
        # Apply attention mechanism
        attention_weights = self.attention(features)
        attended_features = features * attention_weights
        
        # Label prediction
        label_output = self.label_classifier(attended_features)
        
        # Domain prediction with gradient reversal
        reversed_features = GradientReversalLayer.apply(attended_features, alpha)
        domain_output = self.domain_classifier(reversed_features)
        
        return label_output, domain_output

# High-level training strategy

In [6]:
class BalancedBatchSampler:
    """Custom batch sampler for balanced domain training"""
    
    def __init__(self, X1, y1, d1, X2, y2, d2, batch_size):
        self.X1, self.y1, self.d1 = X1, y1, d1
        self.X2, self.y2, self.d2 = X2, y2, d2
        self.batch_size = batch_size
    
    def __iter__(self):
        while True:
            # Sample equal amounts from each domain
            X1_indices = np.random.choice(len(self.X1), self.batch_size // 2, replace=True)
            X2_indices = np.random.choice(len(self.X2), self.batch_size // 2, replace=True)
            
            X_batch = np.vstack([self.X1[X1_indices], self.X2[X2_indices]])
            y_batch = np.hstack([self.y1[X1_indices], self.y2[X2_indices]])
            d_batch = np.hstack([self.d1[X1_indices], self.d2[X2_indices]])
            
            yield torch.FloatTensor(X_batch), torch.LongTensor(y_batch), torch.LongTensor(d_batch)

def evaluate_model(model, X1_val, y1_val, X2_val, y2_val):
    """Evaluate model on validation sets"""
    model.eval()
    with torch.no_grad():
        # Domain 1 validation
        label_out1, _ = model(X1_val)
        pred1 = torch.argmax(label_out1, dim=1)
        acc1 = accuracy_score(y1_val.numpy(), pred1.numpy())
        
        # Domain 2 validation
        label_out2, _ = model(X2_val)
        pred2 = torch.argmax(label_out2, dim=1)
        acc2 = accuracy_score(y2_val.numpy(), pred2.numpy())
        
        # Combined accuracy
        all_preds = torch.cat([pred1, pred2])
        all_labels = torch.cat([y1_val, y2_val])
        combined_acc = accuracy_score(all_labels.numpy(), all_preds.numpy())
    
    model.train()
    return combined_acc

# Train the function

In [7]:
def train_advanced_dann(model, X1_train, y1_train, X2_train, y2_train, 
                       X1_val, y1_val, X2_val, y2_val, 
                       num_epochs=50, batch_size=64, lr=0.001):
    """Advanced training with curriculum learning and adaptive alpha"""
    
    # Prepare domain labels
    d1_train = np.zeros(len(y1_train), dtype=np.int64)  # Domain 1
    d2_train = np.ones(len(y2_train), dtype=np.int64)   # Domain 2
    
    # Convert to tensors
    X1_val_tensor = torch.FloatTensor(X1_val)
    X2_val_tensor = torch.FloatTensor(X2_val)
    y1_val_tensor = torch.LongTensor(y1_val)
    y2_val_tensor = torch.LongTensor(y2_val)
    
    # Initialize optimizer with different learning rates
    optimizer = optim.AdamW([
        {'params': model.feature_extractor.parameters(), 'lr': lr},
        {'params': model.label_classifier.parameters(), 'lr': lr},
        {'params': model.domain_classifier.parameters(), 'lr': lr * 0.1},
        {'params': model.attention.parameters(), 'lr': lr * 0.5}
    ], weight_decay=1e-4)
    
    # Loss functions with class weights
    label_weights = torch.FloatTensor([1.0, 1.0])  # Balanced for labels
    domain_weights = torch.FloatTensor([5.0, 1.0])  # Higher weight for Domain 1
    
    criterion_label = nn.CrossEntropyLoss(weight=label_weights)
    criterion_domain = nn.CrossEntropyLoss(weight=domain_weights)
    
    # Learning rate scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.5)
    
    # Training loop
    model.train()
    batch_sampler = BalancedBatchSampler(X1_train, y1_train, d1_train, X2_train, y2_train, d2_train, batch_size)
    
    best_val_acc = 0.0
    patience_counter = 0
    
    for epoch in range(num_epochs):
        total_loss = 0.0
        total_label_loss = 0.0
        total_domain_loss = 0.0
        
        # Adaptive alpha with curriculum learning
        p = float(epoch) / num_epochs
        alpha = 2.0 / (1.0 + np.exp(-10 * p)) - 1.0
        
        # Training batches per epoch
        batches_per_epoch = max(len(X1_train), len(X2_train)) // batch_size
        
        for batch_idx in range(batches_per_epoch):
            X_batch, y_batch, d_batch = next(iter(batch_sampler))
            
            optimizer.zero_grad()
            
            # Forward pass
            label_output, domain_output = model(X_batch, alpha)
            
            # Compute losses
            label_loss = criterion_label(label_output, y_batch)
            domain_loss = criterion_domain(domain_output, d_batch)
            total_batch_loss = label_loss + domain_loss
            
            # Backward pass
            total_batch_loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            total_loss += total_batch_loss.item()
            total_label_loss += label_loss.item()
            total_domain_loss += domain_loss.item()
        
        # Validation
        if (epoch + 1) % 5 == 0:
            val_acc = evaluate_model(model, X1_val_tensor, y1_val_tensor, X2_val_tensor, y2_val_tensor)
            scheduler.step(val_acc)
            
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/batches_per_epoch:.4f}, "
                  f"Label Loss: {total_label_loss/batches_per_epoch:.4f}, "
                  f"Domain Loss: {total_domain_loss/batches_per_epoch:.4f}, "
                  f"Val Acc: {val_acc:.4f}, Alpha: {alpha:.4f}")
            
            # Early stopping
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                patience_counter = 0
                torch.save(model.state_dict(), 'best_dann_model.pth')
            else:
                patience_counter += 1
                
            if patience_counter >= 10:
                print("Early stopping triggered")
                break
    
    # Load best model
    model.load_state_dict(torch.load('best_dann_model.pth'))
    return model

# Model initializing and train

In [8]:
# Initialize the advanced model
input_dim = X1_train_np.shape[1]
print(f"Input dimension: {input_dim}")

# Model hyperparameters
model = AdvancedDANN(
    input_dim=input_dim,
    hidden_dims=[1024, 512, 256],  # Larger network
    num_classes=2,
    num_domains=2,
    dropout_rate=0.3
)

print(f"Model parameters: {sum(p.numel() for p in model.parameters())}")
print("\nStarting advanced DANN training...")

# Train the model
trained_model = train_advanced_dann(
    model, 
    X1_train_np, y1_train_np, 
    X2_train_np, y2_train_np,
    X1_val_np, y1_val_np,
    X2_val_np, y2_val_np,
    num_epochs=100,
    batch_size=128,
    lr=0.001
)

Input dimension: 15000
Model parameters: 19993861

Starting advanced DANN training...
Epoch [5/100], Loss: 0.7955, Label Loss: 0.0149, Domain Loss: 0.7806, Val Acc: 0.9583, Alpha: 0.1974
Epoch [10/100], Loss: 0.6568, Label Loss: 0.0165, Domain Loss: 0.6404, Val Acc: 0.9642, Alpha: 0.4219
Epoch [15/100], Loss: 0.6234, Label Loss: 0.0220, Domain Loss: 0.6014, Val Acc: 0.9650, Alpha: 0.6044
Epoch [20/100], Loss: 0.6268, Label Loss: 0.0150, Domain Loss: 0.6119, Val Acc: 0.9592, Alpha: 0.7398
Epoch [25/100], Loss: 0.6459, Label Loss: 0.0149, Domain Loss: 0.6310, Val Acc: 0.9667, Alpha: 0.8337
Epoch [30/100], Loss: 0.5801, Label Loss: 0.0156, Domain Loss: 0.5645, Val Acc: 0.9667, Alpha: 0.8957
Epoch [35/100], Loss: 0.6211, Label Loss: 0.0146, Domain Loss: 0.6065, Val Acc: 0.9650, Alpha: 0.9354
Epoch [40/100], Loss: 0.6169, Label Loss: 0.0059, Domain Loss: 0.6110, Val Acc: 0.9658, Alpha: 0.9603
Epoch [45/100], Loss: 0.6085, Label Loss: 0.0129, Domain Loss: 0.5957, Val Acc: 0.9625, Alpha: 0.97

# Evaluate the model

In [9]:
def detailed_evaluation(model, X1, y1, X2, y2, domain_names=['Domain 1', 'Domain 2']):
    """Comprehensive model evaluation"""
    model.eval()
    
    results = {}
    
    for i, (X, y, domain_name) in enumerate([(X1, y1, domain_names[0]), (X2, y2, domain_names[1])]):
        with torch.no_grad():
            X_tensor = torch.FloatTensor(X) if not isinstance(X, torch.Tensor) else X
            y_tensor = torch.LongTensor(y) if not isinstance(y, torch.Tensor) else y
            
            label_output, domain_output = model(X_tensor)
            predictions = torch.argmax(label_output, dim=1)
            domain_predictions = torch.argmax(domain_output, dim=1)
            
            accuracy = accuracy_score(y_tensor.numpy(), predictions.numpy())
            report = classification_report(y_tensor.numpy(), predictions.numpy(), 
                                         target_names=['Human', 'Machine'], output_dict=True)
            
            # Domain classification accuracy
            domain_labels = torch.full_like(y_tensor, i)  # 0 for domain 1, 1 for domain 2
            domain_acc = accuracy_score(domain_labels.numpy(), domain_predictions.numpy())
            
            results[domain_name] = {
                'accuracy': accuracy,
                'classification_report': report,
                'domain_confusion': 1 - domain_acc  # Lower is better (more domain-invariant)
            }
            
            print(f"\n{domain_name} Results:")
            print(f"Label Classification Accuracy: {accuracy:.4f}")
            print(f"Domain Confusion Rate: {1-domain_acc:.4f} (lower is better)")
            print("\nClassification Report:")
            print(classification_report(y_tensor.numpy(), predictions.numpy(), 
                                       target_names=['Human', 'Machine']))
    
    return results

# Evaluate on validation sets
print("=== Final Model Evaluation ===")
val_results = detailed_evaluation(trained_model, X1_val_np, y1_val_np, X2_val_np, y2_val_np)

# Calculate overall metrics
overall_acc = (val_results['Domain 1']['accuracy'] + val_results['Domain 2']['accuracy']) / 2
domain_invariance = (val_results['Domain 1']['domain_confusion'] + val_results['Domain 2']['domain_confusion']) / 2

print(f"\n=== Overall Performance ===")
print(f"Average Accuracy: {overall_acc:.4f}")
print(f"Domain Invariance Score: {domain_invariance:.4f}")
print(f"Balanced Score: {overall_acc * (1 + domain_invariance):.4f}")

=== Final Model Evaluation ===

Domain 1 Results:
Label Classification Accuracy: 0.9100
Domain Confusion Rate: 0.0000 (lower is better)

Classification Report:
              precision    recall  f1-score   support

       Human       0.96      0.86      0.91       100
     Machine       0.87      0.96      0.91       100

    accuracy                           0.91       200
   macro avg       0.91      0.91      0.91       200
weighted avg       0.91      0.91      0.91       200


Domain 2 Results:
Label Classification Accuracy: 0.9810
Domain Confusion Rate: 1.0000 (lower is better)

Classification Report:
              precision    recall  f1-score   support

       Human       0.88      0.72      0.79        50
     Machine       0.99      0.99      0.99       950

    accuracy                           0.98      1000
   macro avg       0.93      0.86      0.89      1000
weighted avg       0.98      0.98      0.98      1000


=== Overall Performance ===
Average Accuracy: 0.9455
Dom

# Test the data predictions and make the submission

In [10]:
# Load and process test data
print("Loading test data...")
X_test_raw, test_ids = load_test_data('test_data.json')

# Transform test data using the same vectorizers
X_test_count = count_vectorizer.transform(X_test_raw)
X_test_tfidf = tfidf_vectorizer.transform(X_test_raw)
X_test_combined = hstack([X_test_count, X_test_tfidf])
X_test_tensor = torch.FloatTensor(X_test_combined.toarray())

print(f"Test data shape: {X_test_tensor.shape}")

# Generate predictions
trained_model.eval()
with torch.no_grad():
    label_output, _ = trained_model(X_test_tensor)
    probabilities = F.softmax(label_output, dim=1)
    predictions = torch.argmax(label_output, dim=1).tolist()
    confidence_scores = torch.max(probabilities, dim=1)[0].tolist()

# Create submission file
submission_df = pd.DataFrame({
    'id': test_ids,
    'class': predictions
})

# Save predictions
submission_df.to_csv('advanced_dann_predictions.csv', index=False)
print(f"\nPredictions saved to 'advanced_dann_predictions.csv'")
print(f"Prediction distribution: {np.bincount(predictions)}")
print(f"Average confidence: {np.mean(confidence_scores):.4f}")

# Display first few predictions
print("\nFirst 10 predictions:")
print(submission_df.head(10))

Loading test data...
Test data shape: torch.Size([4000, 15000])

Predictions saved to 'advanced_dann_predictions.csv'
Prediction distribution: [1073 2927]
Average confidence: 0.9933

First 10 predictions:
   id  class
0   0      1
1   1      0
2   2      0
3   3      1
4   4      0
5   5      1
6   6      0
7   7      1
8   8      1
9   9      1
